In [1]:
import keras
import tensorflow as tf
import keras.backend as K
from keras.models import Model
import warnings
warnings.filterwarnings("ignore")

from models import matting_net

def convert_checkpoint_into_tflite(h5_path=None,
                                   output_path=None,
                                   android=True):
    if android:
        model = matting_net((256,256,4), android=True)
        
    else:
        model = matting_net((256,256,3), android=False)
    # model 명 과 상세사항 정리해두기 
    model.load_weights(h5_path, by_name=True)

    input_names = [node.op.name for node in model.inputs]
    output_names = [node.op.name for node in model.outputs]
    print(input_names)
    print(output_names)

    sess = K.get_session()
    converter = tf.lite.TFLiteConverter.from_session(sess, model.inputs, model.outputs)

    tflite_model = converter.convert()
    open(output_path, "wb").write(tflite_model)
    print("tflite successfully built")
    
    return model

Using TensorFlow backend.


In [ ]:
tf.

In [3]:
tf.random.set_seed()

TypeError: set_seed() missing 1 required positional argument: 'seed'

In [2]:
model = convert_checkpoint_into_tflite(h5_path="./trained_models/20191211/560.h5",
                                       output_path="./tflite/560.tflite", 
                                       android=True)

['input_1']
['output/Sigmoid']


RuntimeError: `get_session` is not available when using TensorFlow 2.0.

## Model output test

In [ ]:
import numpy as np 
import cv2
import matplotlib.pyplot as plt

img = cv2.imread("./dataset/selfie/training/00694.png", cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (256,256))
img = np.concatenate([img, np.ones((256,256,1))], axis=-1)[np.newaxis,:,:,:]
img = img / 255.

mask = cv2.imread("./dataset/selfie/training/00694_matte.png", cv2.IMREAD_GRAYSCALE)
mask = cv2.resize(mask, (256,256))

pred = model.predict(img)

In [ ]:
plt.figure(figsize=(19,7))
plt.subplot(131)
plt.imshow(img.squeeze(0)[:,:,:3])
plt.axis("off")

plt.subplot(132)
plt.imshow(mask)
plt.axis("off")

plt.subplot(133)
plt.imshow(pred.squeeze(0).squeeze(-1) * 255)
plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
def iou_coef(y_true, y_pred, smooth=1):
    """
    IoU = (|X &amp; Y|)/ (|X or Y|)
    """
#     intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    intersection = np.sum(np.abs(y_true * y_pred), axis=-1)
    union = np.sum(y_true) + np.sum(y_pred) - intersection
    
#     union = K.sum(y_true,-1) + K.sum(y_pred,-1) - intersection
    return (intersection + smooth) / (union + smooth)

In [ ]:
iou_coef(mask[np.newaxis, :, :, np.newaxis], pred)

In [ ]:
from sklearn.metrics import confusion_matrix  

def compute_iou(y_pred, y_true):
     # ytrue, ypred is a flatten vector
    y_pred = y_pred.flatten()
    y_true = y_true.flatten()
    current = confusion_matrix(y_true, y_pred, labels=[0, 1])
    # compute mean iou
    intersection = np.diag(current)
    ground_truth_set = current.sum(axis=1)
    predicted_set = current.sum(axis=0)
    union = ground_truth_set + predicted_set - intersection
    IoU = intersection / union.astype(np.float32)
    return np.mean(IoU)

In [ ]:
mask_ = cv2.threshold(mask, 255/2, 255, cv2.THRESH_BINARY)
pred_ = cv2.threshold(pred.squeeze(0).squeeze(-1), 0.5, 1, cv2.THRESH_BINARY)
compute_iou(mask_[1], pred_[1])

In [ ]:
import numpy as np
from sklearn.metrics import log_loss

def cross_entropy(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    
    N = y_pred.shape[0]
    
    ce = -np.sum(y_true*np.log(y_pred + 1e-5)) / N
    
    return ce


ce = cross_entropy(mask, pred.squeeze(0).squeeze(-1))

ce

In [ ]:
mask, pred_

In [ ]:
train_params = {
            'dim': (256,256),
            'batch_size': 32,
            'n_channels': 3,
            'shuffle': True,
            'augment': True,
        }

In [ ]:
img_paths = np.load("./dataset/selfie/img_paths.npy")

train_img_paths = np.random.choice(img_paths, int(img_paths.shape[0] * 0.8), replace=False)

test_img_paths = np.setdiff1d(img_paths, train_img_paths)

In [ ]:
from data_generator import DataGeneratorMatting
train_gen = DataGeneratorMatting(train_img_paths, **train_params)

In [ ]:
train_gen[0][1]

In [ ]:
preds = model.predict(np.concatenate((train_gen[0][0], np.ones((32, 256, 256, 1))), axis=-1))

In [ ]:
preds.shape

In [ ]:
for pr in preds[:10]:
    plt.imshow(pr.squeeze(-1))
    plt.show()

In [ ]:
np.unique(pr)

In [ ]:
train_gen[0][0].shape

In [ ]:
np.ones((32, 256, 256, 1)).shape